## Importing Libraries

In [1]:
import os
import pandas as pd
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

import IPython.display as ipd
import librosa
import librosa.display

## Defining Constants

In [33]:
path = "/Users/priyankar83/Documents/Learning/Data_Science_Learning/Deep_Learning/Audio_Classification/Data/"
Model_path = "/Users/priyankar83/Documents/Learning/Data_Science_Learning/Deep_Learning/Audio_Classification/Model/"
audio_file = "UrbanSound8K/audio/"
audio_dataset_path = path+audio_file
metadata = "UrbanSound8K/metadata/UrbanSound8K.csv"

num_epochs = 100
num_batch_size = 32

In [3]:
meta_data = pd.read_csv(path+metadata)

meta_data.shape

(8732, 8)

In [4]:
meta_data.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


## Creating Features

In [5]:
def features_extractor(file):
    audio, sample_rate = librosa.load(file_name, 
                                      res_type = "kaiser_fast")
    mfccs_features = librosa.feature.mfcc(y = audio, 
                                          sr = sample_rate,
                             # n_mfcc is a hyperparameter, we can try different different values such as 30, 50, 60 etc.
                                          n_mfcc = 40)
    mfccs_scaled_features = np.mean(mfccs_features.T, axis = 0)
    
    return (mfccs_scaled_features)

## Iterating through every audio file for feature extraction using MFCC

In [6]:
extracted_features = []

for index_num, row in tqdm(meta_data.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_dataset_path), "fold"+str(row["fold"])+"/", str(row["slice_file_name"]))
    final_class_labels = row["class"]
    data = features_extractor(file_name)
    extracted_features.append([data, final_class_labels])

3554it [02:09, 28.83it/s]/Users/priyankar83/anaconda3/envs/tensorflow/lib/python3.11/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1323
  warnings.warn(
8324it [04:44, 41.35it/s]/Users/priyankar83/anaconda3/envs/tensorflow/lib/python3.11/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1103
  warnings.warn(
/Users/priyankar83/anaconda3/envs/tensorflow/lib/python3.11/site-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=2048 is too large for input signal of length=1523
  warnings.warn(
8732it [04:57, 29.35it/s]


## Converting the extracted features to DataFrame

In [7]:
extracted_features_df = pd.DataFrame(data = extracted_features, columns = ["feature", "class"])

extracted_features_df.shape

(8732, 2)

In [8]:
extracted_features_df.head()

,feature,class
0,"[-217.35526, 70.22338, -130.38527, -53.282898,...",dog_bark
1,"[-424.09818, 109.34077, -52.919525, 60.86475, ...",children_playing
2,"[-458.79114, 121.38419, -46.520657, 52.00812, ...",children_playing
3,"[-413.89984, 101.66373, -35.42945, 53.036358, ...",children_playing
4,"[-446.60352, 113.68541, -52.402206, 60.302044,...",children_playing


## Splitting the dataset into independent and dependent features

In [20]:
X = np.array(extracted_features_df["feature"].values.tolist())
y = np.array(extracted_features_df["class"].values.tolist())

print(X.shape, y.shape)

(8732, 40) (8732,)


## Encoding the target

In [21]:
y = np.array(pd.get_dummies(y, dtype = int))

In [22]:
y.shape

(8732, 10)

In [23]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0]])

## Splitting the dataset into Train and Test

In [24]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(6985, 40) (1747, 40) (6985, 10) (1747, 10)


## Finding number of classes in Target variable

In [26]:
num_labels = y.shape[1]

In [27]:
num_labels

10

## Model Creation

In [25]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

## ANN Model

In [30]:
model = Sequential()
#First layer
model.add(Dense(units = 100, input_shape = (40,)))
model.add(Activation("relu"))
model.add(Dropout(0.5))
#Second layer
model.add(Dense(units = 200))
model.add(Activation("relu"))
model.add(Dropout(0.5))
#Third layer
model.add(Dense(units = 100))
model.add(Activation("relu"))
model.add(Dropout(0.5))
#Final layer
model.add(Dense(num_labels))
model.add(Activation("softmax"))

## Model Summary

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 100)               4100      
                                                                 
 activation (Activation)     (None, 100)               0         
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 200)               20200     
                                                                 
 activation_1 (Activation)   (None, 200)               0         
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_3 (Dense)             (None, 100)              

## Model Compilation

In [32]:
model.compile(loss = "categorical_crossentropy", metrics = ["accuracy"], optimizer = "adam")

## Model Training

In [38]:
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime

checkpointer = ModelCheckpoint(filepath = Model_path+"audio_classification.hdf5", 
                               verbose =1, 
                               save_best_only = True)

start = datetime.now()

model.fit(X_train, y_train, 
          batch_size = num_batch_size, 
          epochs = num_epochs, 
          validation_data = (X_test, y_test),
          callbacks = [checkpointer],
          verbose = 1)

duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
206/219 [===========================>..] - ETA: 0s - loss: 0.9562 - accuracy: 0.6883
Epoch 1: val_loss improved from inf to 0.70684, saving model to /Users/priyankar83/Documents/Learning/Data_Science_Learning/Deep_Learning/Audio_Classification/Model/audio_classification.hdf5
219/219 [==============================] - 0s 1ms/step - loss: 0.9545 - accuracy: 0.6885 - val_loss: 0.7068 - val_accuracy: 0.7916
Epoch 2/100
217/219 [============================>.] - ETA: 0s - loss: 0.9420 - accuracy: 0.6961
Epoch 2: val_loss improved from 0.70684 to 0.69705, saving model to /Users/priyankar83/Documents/Learning/Data_Science_Learning/Deep_Learning/Audio_Classification/Model/audio_classification.hdf5
219/219 [==============================] - 0s 1ms/step - loss: 0.9409 - accuracy: 0.6961 - val_loss: 0.6971 - val_accuracy: 0.7962
Epoch 3/100
217/219 [============================>.] - ETA: 0s - loss: 0.9258 - accuracy: 0.6921
Epoch 3: val_loss did not improve from 0.69705
219/219 [=====

219/219 [==============================] - 0s 1ms/step - loss: 0.9026 - accuracy: 0.7028 - val_loss: 0.6831 - val_accuracy: 0.8002
Epoch 28/100
218/219 [============================>.] - ETA: 0s - loss: 0.9211 - accuracy: 0.7020
Epoch 28: val_loss did not improve from 0.66916
219/219 [==============================] - 0s 1ms/step - loss: 0.9220 - accuracy: 0.7015 - val_loss: 0.6696 - val_accuracy: 0.8031
Epoch 29/100
168/219 [======================>.......] - ETA: 0s - loss: 0.9145 - accuracy: 0.6940
Epoch 29: val_loss did not improve from 0.66916
219/219 [==============================] - 0s 1ms/step - loss: 0.9298 - accuracy: 0.6911 - val_loss: 0.6805 - val_accuracy: 0.8002
Epoch 30/100
166/219 [=====================>........] - ETA: 0s - loss: 0.9109 - accuracy: 0.6875
Epoch 30: val_loss did not improve from 0.66916
219/219 [==============================] - 0s 1ms/step - loss: 0.9127 - accuracy: 0.6932 - val_loss: 0.6795 - val_accuracy: 0.8037
Epoch 31/100
166/219 [================

219/219 [==============================] - 0s 1ms/step - loss: 0.8626 - accuracy: 0.7164 - val_loss: 0.6362 - val_accuracy: 0.8042
Epoch 54/100
165/219 [=====================>........] - ETA: 0s - loss: 0.8593 - accuracy: 0.7155
Epoch 54: val_loss did not improve from 0.63094
219/219 [==============================] - 0s 1ms/step - loss: 0.8706 - accuracy: 0.7144 - val_loss: 0.6611 - val_accuracy: 0.7985
Epoch 55/100
167/219 [=====================>........] - ETA: 0s - loss: 0.8519 - accuracy: 0.7176
Epoch 55: val_loss did not improve from 0.63094
219/219 [==============================] - 0s 1ms/step - loss: 0.8679 - accuracy: 0.7148 - val_loss: 0.6345 - val_accuracy: 0.8174
Epoch 56/100
166/219 [=====================>........] - ETA: 0s - loss: 0.8715 - accuracy: 0.7135
Epoch 56: val_loss did not improve from 0.63094
219/219 [==============================] - 0s 1ms/step - loss: 0.8774 - accuracy: 0.7114 - val_loss: 0.6417 - val_accuracy: 0.8105
Epoch 57/100
218/219 [================

219/219 [==============================] - 0s 1ms/step - loss: 0.8663 - accuracy: 0.7174 - val_loss: 0.6436 - val_accuracy: 0.8088
Epoch 81/100
217/219 [============================>.] - ETA: 0s - loss: 0.8395 - accuracy: 0.7213
Epoch 81: val_loss did not improve from 0.61147
219/219 [==============================] - 0s 1ms/step - loss: 0.8392 - accuracy: 0.7211 - val_loss: 0.6526 - val_accuracy: 0.8031
Epoch 82/100
167/219 [=====================>........] - ETA: 0s - loss: 0.8456 - accuracy: 0.7208
Epoch 82: val_loss did not improve from 0.61147
219/219 [==============================] - 0s 1ms/step - loss: 0.8593 - accuracy: 0.7178 - val_loss: 0.6306 - val_accuracy: 0.8019
Epoch 83/100
167/219 [=====================>........] - ETA: 0s - loss: 0.8623 - accuracy: 0.7169
Epoch 83: val_loss did not improve from 0.61147
219/219 [==============================] - 0s 1ms/step - loss: 0.8691 - accuracy: 0.7137 - val_loss: 0.6197 - val_accuracy: 0.8140
Epoch 84/100
166/219 [================

In [39]:
accuracy = model.evaluate(X_test, y_test, verbose = 0)
print(accuracy[1])

0.8093875050544739
